In [1]:
import unicodecsv

In [2]:
with open('diagnosis.csv', 'rb') as f:
    reader = unicodecsv.DictReader(f)

In [163]:
for each in reader:
    print(each)

ValueError: readline of closed file

In [3]:
diagnosis = []
f = open('breast.csv', 'rb')
reader = unicodecsv.DictReader(f)
for row in reader:
    diagnosis.append(row)
f.close()

In [4]:
print(len(diagnosis))
diagnosis[23]

699


OrderedDict([('id', '1057013'),
             ('clump', '8'),
             ('size', '4'),
             ('shape', '5'),
             ('adhesion', '1'),
             ('epithelial', '2'),
             ('barenuclei', '?'),
             ('bland', '7'),
             ('normal', '3'),
             ('mitoses', '1'),
             ('class', '4')])

In [5]:
def check(data):
    for v in data.values():
        if v=='?':
            return False
    return True

In [6]:
cleanedData=[]
for diag in diagnosis:
    if check(diag):
        diag.pop('id')
        cleanedData.append(diag)
    

In [131]:
len(cleanedData)

683

In [62]:
class Node:
    def __init__(self,testAttr,predictedClass):
        self.testAttr=testAttr
        self.predictedClass=predictedClass
        self.children={}
    def __str__(self):
        return self.testAttr
def printTree(n):
    print(n)
    if len(n.children)==0:
        return
    print('children are:')
    for k,v in n.children.items():
        print(k,v)
    for v in n.children.values():
        printTree(v)

In [63]:
with open('breastAttr.txt','r') as f:
    breastAttr=f.read()

In [64]:
s=breastAttr.split('\n')

In [65]:
possVals

{'adhesion': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
 'barenuclei': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
 'bland': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
 'clump': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
 'epithelial': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
 'mitoses': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
 'normal': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
 'shape': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
 'size': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']}

In [66]:
possVals={}
for line in s:
    temp=line.split(' ')
    possVals[temp[0]]=temp[1].split(',')

In [67]:
possClassValues=[]
with open('breastClass.txt','r') as f:
    breastClass=f.read()
classLine=breastClass.split(' ')
className = classLine[0]
possClassValues = classLine[1].split(',')

In [68]:
possClassValues,className

(['2', '4'], 'class')

In [69]:
prob={}

In [70]:
from collections import defaultdict
from math import log2

In [71]:
def entropy(edata):
    temp = defaultdict(int)
    for eavpair in edata:
        temp[eavpair[className]]+=1
    en = len(edata)
    e=1
    for epv in temp.values():
        e*=(epv/en)*log2(epv/en)
    return e

In [149]:
from math import cos,sqrt,log2
def balance(attrLen):
    ans=cos(3.5*attrLen-1.5)/1.8
    ans*=log2(sqrt(attrLen+1))
    return abs(ans)

In [150]:
def train(trainData,attributes):
    e = entropy(trainData)
    info = {}
    gain = {}
    n = len(trainData)
    for attr in attributes:
        attrVals = possVals[attr]
        attrTrainData = {}
        attrTrainData = defaultdict(list)
        for avpair in trainData:
            attrTrainData[avpair[attr]].append(avpair)
        info[attr] = 1
        for attrVal in attrTrainData.values():
            info[attr] += (len(attrVal)/n)*entropy(attrVal)
            gain[attr] = (e-info[attr])/balance(len(attrVals))
    maxAttr = attributes[0]
    maxGain = gain[maxAttr]
    for k,v in gain.items():
        if maxGain < v:
            maxGain = v
            maxAttr = k
    print('maxAttr is {0}'.format(maxAttr))

In [151]:
train(cleanedData,list(possVals.keys()))

maxAttr is size


In [163]:
def train(trainData,attributes):
    e = entropy(trainData)
    if e==0:
        return Node('',trainData[0][className])
    if len(attributes) == 0:
        classCount = defaultdict(int)
        for ex in trainData:
            classCount[ex[className]] += 1
        maxClass= possClassValues[0]
        maxCount= classCount[maxClass]
        for k,v in classCount.items():
            if maxCount<v:
                maxCount = v
                maxClass = k
        return Node('',maxClass)
    info = {}
    gain = {}
    n = len(trainData)
    for attr in attributes:
        attrVals = possVals[attr]
        attrTrainData = {}
        attrTrainData = defaultdict(list)
        for avpair in trainData:
            attrTrainData[avpair[attr]].append(avpair)
        info[attr] = 1
        for attrVal in attrTrainData.values():
            info[attr] += (len(attrVal)/n)*entropy(attrVal)
            gain[attr] = (e-info[attr])/balance(len(attrVals))
    maxAttr = attributes[0]
    maxGain = gain[maxAttr]
    for k,v in gain.items():
        if maxGain < v:
            maxGain = v
            maxAttr = k
    n=Node(maxAttr,'')
    splitTrainData = defaultdict(list)
    for ex in trainData:
        splitTrainData[ex[maxAttr]].append(ex)
    newAttrs = attributes[:]
    newAttrs.remove(maxAttr)
    for splitAttrVal, splitData in splitTrainData.items():
        n.children[splitAttrVal]=train(splitData,newAttrs)
    return n

In [164]:
n=train(cleanedData[:200],list(possVals.keys()))

In [165]:
for k,v in n.children.items():
    print(k,v)

1 
4 adhesion
8 clump
10 
2 clump
3 barenuclei
5 
6 
7 
9 


In [166]:
def testing(n,ex):
    while n.testAttr!='':
        child=ex[n.testAttr]
        n=n.children[child]
    return n.predictedClass

In [167]:
print(testing(n,cleanedData[1]))

2


In [171]:

def testing1(n,trainData):
    x=0
    for ex in trainData:
        p=n
        try:
            while p.testAttr!='':
                child=ex[p.testAttr]
                p=p.children[child]
            print(p.predictedClass,ex[className])
            if p.predictedClass==ex[className]:
                x+=1
        except:
            oo=2
    return x

In [172]:
x=testing1(n,cleanedData[501:])

2 2
2 2
2 2
2 2
2 2
4 4
4 4
2 2
2 2
2 2
2 2
2 2
2 2
4 4
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
4 4
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
4 4
2 2
2 2
4 4
4 4
4 4
4 4
2 2
2 2
4 4
2 2
2 2
2 2
2 2
4 4
4 4
2 2
2 2
2 2
4 4
2 2
4 4
2 2
4 4
2 4
4 4
2 2
2 2
2 2
2 2
4 2
2 2
2 2
2 2
4 4
4 4
2 2
2 2
4 4
2 2
4 4
4 4
4 4
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
4 2
2 2
2 2
4 2
4 4
2 2
2 2
2 2
2 2
2 2
2 2
4 4
2 2
4 4
4 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
4 4
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
4 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
4 4
4 4
4 4
2 2
2 2
2 2
2 2
2 2
2 2
2 2
4 4
4 4
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
4 4
2 2
2 2
2 2
2 2
4 4
4 4


In [173]:
print(x)

161


In [160]:
import math

In [148]:
abs?